In [ ]:
import pandas as pd
import numpy as np
import os, sys
import tqdm
import matplotlib.pyplot as plt

trainLabelPath = '../../hfactory_magic_folders/colas_data_challenge/computer_vision_challenge/dataset/labels_train.csv'
trainLabels = pd.read_csv(trainLabelPath)

In [ ]:
categories = trainLabels.columns[1:6]
y = trainLabels[categories].values.tolist()

In [ ]:
imageTrainPath = '../../hfactory_magic_folders/colas_data_challenge/computer_vision_challenge/dataset/train/'

In [ ]:
import torch
import numpy as np
import cv2
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from keras.preprocessing import image

class ImageDataset(Dataset):
    def __init__(self, csv, train: bool, test: bool) -> Dataset():
        self.csv = csv
        self.train = train
        self.test = test
        self.all_image_names = self.csv[:]['filename']
        self.all_labels = np.array(self.csv.drop(['filename'], axis=1))
        self.train_ratio = int(0.8 * len(self.csv))
        self.valid_ratio = len(self.csv) - self.train_ratio
        # set the training data images and labels
        if self.train == True:
            print(f"Number of training images: {self.train_ratio}")
            self.image_names = list(self.all_image_names[:self.train_ratio])
            self.labels = list(self.all_labels[:self.train_ratio])
            # define the training transforms
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((400, 400)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(degrees=45),
                transforms.ToTensor(),
            ])
        # set the validation data images and labels
        elif self.train == False:
            print(f"Number of validation images: {self.valid_ratio}")
            self.image_names = list(self.all_image_names[-self.valid_ratio:-10])
            self.labels = list(self.all_labels[-self.valid_ratio:])
            # define the validation transforms
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((400, 400)),
                transforms.ToTensor(),
            ])

    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, index):
        #img = image.load_img(imageTrainPath+self.image_names[index], target_size=(400,400,3))
        image = cv2.imread(imageTrainPath+self.image_names[index])
        # convert the image from BGR to RGB color format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # apply image transforms
        image = self.transform(image)
        targets = self.labels[index]
        
        return {
            'image': torch.tensor(image, dtype=torch.float32),
            'label': torch.tensor(targets, dtype=torch.float32)
        }

In [ ]:
from torchvision import models as models
import torch.nn as nn

def model(pretrained, requires_grad):
    model = models.resnet50(progress=True, pretrained=pretrained)
    # to freeze the hidden layers
    if requires_grad == False:
        for param in model.parameters():
            param.requires_grad = False
    # to train the hidden layers
    elif requires_grad == True:
        for param in model.parameters():
            param.requires_grad = True
    # make the classification layer learnable
    # we have 5 classes in total
    model.fc = nn.Linear(2048, 5)
    return model

In [ ]:
import torch
from tqdm import tqdm
# training function
def train(model, dataloader, optimizer, criterion, trainData, device):
    print('Training')
    model.train()
    counter = 0
    train_running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(trainData)/dataloader.batch_size)):
        counter += 1
        data, target = data['image'].to(device), data['label'].to(device)
        optimizer.zero_grad()
        outputs = model(data)
        # apply sigmoid activation to get all the outputs between 0 and 1
        outputs = torch.sigmoid(outputs)
        loss = criterion(outputs, target)
        train_running_loss += loss.item()
        # backpropagation
        loss.backward()
        # update optimizer parameters
        optimizer.step()
        
    train_loss = train_running_loss / counter
    return train_loss

# validation function
def validate(model, dataloader, criterion, valData, device):
    print('Validating')
    model.eval()
    counter = 0
    val_running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(valData)/dataloader.batch_size)):
            counter += 1
            data, target = data['image'].to(device), data['label'].to(device)
            outputs = model(data)
            # apply sigmoid activation to get all the outputs between 0 and 1
            outputs = torch.sigmoid(outputs)
            loss = criterion(outputs, target)
            val_running_loss += loss.item()
        
        val_loss = val_running_loss / counter
        return val_loss

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from torch.utils.data import DataLoader
matplotlib.style.use('ggplot')
# initialize the computation device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#intialize the model
model = model(pretrained=True, requires_grad=False).to(device)
# learning parameters
lr = 0.0001
epochs = 20
batch_size = 32
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

In [ ]:
trainData = ImageDataset(trainLabels, train=True, test=False)
valData = ImageDataset(trainLabels, train=False, test=False)
batchSize = 16
# train data loader
trainLoader = DataLoader(
    trainData, 
    batch_size=batchSize,
    shuffle=True
)
# validation data loader
validLoader = DataLoader(
    valData, 
    batch_size=batchSize,
    shuffle=False)

In [ ]:
train_loss = []
valid_loss = []
epochs = 1
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = train(
        model, trainLoader, optimizer, criterion, trainData, device
    )
    valid_epoch_loss = validate(
        model, validLoader, criterion, valData, device
    )
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f'Val Loss: {valid_epoch_loss:.4f}')

In [ ]:
model.eval()

In [ ]:
val